## Install

In [1]:
from IPython.display import clear_output
#install
!pip install -U transformers
!pip install -U huggingface-hub
!pip install -U langchain
!pip install -U accelerate
!pip install -U datasets

# not exit in python 3.11
#!pip install -U intel_extension_for_pytorch

!pip install -U bitsandbytes
# use in windows machine. !!!May not be safe!!!
#!pip install -U bitsandbytes --prefer-binary --extra-index-url=https://jllllll.github.io/bitsandbytes-windows-webui
!pip install -U peft
!pip install -U trl
!pip install -U pprintpp


# install FlashAttention 2
# !pip install -U ninja
# !pip install -U packaging
# !MAX_JOBS=4 pip install -U flash-attn --no-build-isolation # if error -> use pip install -U flash-attn

!pip install -U streamlit
!pip install -U langchain-experimental
clear_output()

## Colab

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
%cd "/content/drive/MyDrive/Colab Notebooks/zephyr"

/content/drive/MyDrive/Colab Notebooks/zephyr


## Import

In [11]:
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
    pipeline,
    Conversation,
)
from langchain.llms.huggingface_pipeline import HuggingFacePipeline
from langchain import PromptTemplate, LLMChain
import numpy as np
import pandas as pd
import torch
from datasets import Dataset, load_dataset
from peft import LoraConfig, PeftModel
from trl import SFTTrainer
from sklearn.model_selection import train_test_split

device = "cuda" if torch.cuda.is_available() else "cpu"
cache_dir="./Model/"
model_name="HuggingFaceH4/zephyr-7b-beta"

import locale
locale.getpreferredencoding = lambda: "UTF-8"


## Model

In [12]:
class Chatbot():
  def __init__(self,model_name = "HuggingFaceH4/zephyr-7b-beta",cache_dir="./Model/",max_vram = 7):
    self.device = "cuda" if torch.cuda.is_available() else "cpu"
    self.model_name = model_name
    self.cache_dir = cache_dir
    self.quantization_config = BitsAndBytesConfig( load_in_4bit=True,
                                                  bnb_4bit_compute_dtype=torch.bfloat16,
                                                  bnb_4bit_use_double_quant=True,
                                                  bnb_4bit_quant_type='nf4')
    self.tokenizer = AutoTokenizer.from_pretrained(self.model_name,cache_dir=self.cache_dir)
    self.tokenizer.pad_token = self.tokenizer.eos_token
    self.tokenizer.padding_side = "right"
    self.pipe=None
    self.langchain_hf=None
    self.max_vram = max_vram

  def InitChat(self,lora=None):
    torch.cuda.empty_cache()
    self.model = AutoModelForCausalLM.from_pretrained( model_name,
                                                      cache_dir=self.cache_dir,
                                                      low_cpu_mem_usage=True,
                                                      return_dict=True,
                                                      quantization_config=self.quantization_config,
                                                      # if error -> use use_flash_attention_1
                                                      #use_flash_attention_2=True,
                                                      max_memory={0: f"{self.max_vram}GB"},
                                                      device_map="auto",
                                                      torch_dtype=torch.bfloat16,
      )
    if lora != None:
      self.model = PeftModel.from_pretrained(self.model, lora)
      self.model = self.model.merge_and_unload()
    self.pipe = pipeline("text-generation", model=self.model,tokenizer=self.tokenizer)

  def InitLangChain(self,max_new_tokens = 4096, temperature=0.6, top_k=50, top_p=0.95):
    if self.pipe==None:
      self.InitChat()
    self.pipe = pipeline("text-generation", model=self.model,tokenizer=self.tokenizer,
                         do_sample=True,max_new_tokens = max_new_tokens, temperature=temperature, top_k=top_k, top_p=top_p)
    self.langchain_hf = HuggingFacePipeline(pipeline=self.pipe)
  def CreatConversation(self):
    return  [
        {
            "role": "system",
            "content": "You are a wonderful and trustworthy assistant. If you receive a request in Vietnamese, please respond in Vietnamese.",
        }
    ]
  def __AddUserInput(self,content,messages):
     messages.append({"role": "user", "content": f"{content.strip()}"})
     return messages

  def __AddAssistantOutput(self,content,messages):
    temp = (content.split("<|assistant|>"))[-1].strip()
    messages.append({"role": "assistant", "content": f"{temp}"})
    return messages

  def Chat(self,content:str ="",messages = None,max_new_tokens = 4096,temperature=0.6, top_k=50, top_p=0.95):
    torch.cuda.empty_cache()
    if len(content)==0:
      return None
    if messages == None:
      messages = self.CreatConversation()

    messages = self.__AddUserInput(content,messages)
    return self._Generate(messages,max_new_tokens,temperature, top_k, top_p)

  def _Generate(self,messages,max_new_tokens = 4096, temperature=0.6, top_k=50, top_p=0.95):
    if self.pipe==None:
      self.InitChat()
    # use_flash_attention_1 if use_flash_attention_2 error
    with torch.backends.cuda.sdp_kernel(enable_flash=True, enable_math=False, enable_mem_efficient=False):
      prompt = self.pipe.tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
      outputs = self.pipe(prompt, max_new_tokens=max_new_tokens, do_sample=True, temperature=temperature, top_k=top_k, top_p=top_p)
      messages = self.__AddAssistantOutput(outputs[0]["generated_text"],messages)
    return messages

  def GetLastContent(self,messages):
    return (messages[-1]["content"])

## Test Model

In [ ]:
chat = Chatbot()
chat.InitChat()

Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

In [ ]:
messages = chat.Chat("Xin chào tôi cần giúp đỡ",max_new_tokens = 4096,temperature=0.6, top_k=50, top_p=0.95)
print(chat.GetLastContent(messages))

Xin chào, tôi cá nhận rất vui với việc giúp bạn. Vui lòng để lại yêu cầu của bạn ở đây, tôi sẽ trả lời ngay.


In [ ]:
messages = chat.Chat("Một năm có bao nhiêu ngày",messages,max_new_tokens = 4096,temperature=0.6, top_k=50, top_p=0.95)
print(chat.GetLastContent(messages))

Một năm có 365 ngày (trừ năm nhuận đơn) trong khoảng giữa hai ngày của tháng 12 và tháng 0 (các môi trường khác có thể sử dụng hệ thống năm khác như 360 ngày trong một năm).


In [ ]:
messages = chat.Chat("Gải thích thuật ngữ \"độc quyền\"",messages,max_new_tokens = 4096,temperature=0.6, top_k=50, top_p=0.95)
print(chat.GetLastContent(messages))

Độc quyền (copyright) là quyền tác giả hoặc tác giả có quyền sở hữu được đưa ra khi tạo ra một tập lược, hình ảnh, khám phá, động thái, văn bản, hoặc không gian bản quyền khác. Nó cho phép chủ sở hữu có quyền tự do để chọn các cách sử dụng, phân phối, sản xuất, hoặc bán của bản quyền đó. Độc quyền cũng có thể được truyền qua không gian bản quyền sau khi chủ sở hữu chết, nhưng thời gian nà có thể khác tùy thể sự nghiệp của chủ sở hữu.


In [ ]:
messages = chat.Chat("Cảm ơn thông tin rất hữu ích",messages,max_new_tokens = 4096,temperature=0.6, top_k=50, top_p=0.95)
print(chat.GetLastContent(messages))

Xin hãy không heesitate nếu có bất kỳ thắc mắc nà khác, tôi sẽ cố gắng cung cấp các thông tin chính xác nhất có sẵn.


In [ ]:
print(messages)

[{'role': 'system', 'content': 'You are a wonderful and trustworthy assistant. If you receive a request in Vietnamese, please respond in Vietnamese.'}, {'role': 'user', 'content': 'Xin chào tôi cần giúp đỡ'}, {'role': 'assistant', 'content': 'Xin chào, tôi cá nhận rất vui với việc giúp bạn. Vui lòng để lại yêu cầu của bạn ở đây, tôi sẽ trả lời ngay.'}, {'role': 'user', 'content': 'Một năm có bao nhiêu ngày'}, {'role': 'assistant', 'content': 'Một năm có 365 ngày (trừ năm nhuận đơn) trong khoảng giữa hai ngày của tháng 12 và tháng 0 (các môi trường khác có thể sử dụng hệ thống năm khác như 360 ngày trong một năm).'}, {'role': 'user', 'content': 'Gải thích thuật ngữ "độc quyền"'}, {'role': 'assistant', 'content': 'Độc quyền (copyright) là quyền tác giả hoặc tác giả có quyền sở hữu được đưa ra khi tạo ra một tập lược, hình ảnh, khám phá, động thái, văn bản, hoặc không gian bản quyền khác. Nó cho phép chủ sở hữu có quyền tự do để chọn các cách sử dụng, phân phối, sản xuất, hoặc bán của bản

### Using LangChain

In [ ]:
chat = Chatbot(max_vram=13)
chat.InitLangChain(max_new_tokens = 4096, temperature=0.6, top_k=50, top_p=0.95)

Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

### Pandas

In [ ]:
from langchain.agents.agent_types import AgentType
from langchain_experimental.agents.agent_toolkits import create_pandas_dataframe_agent
import pandas as pd


data = pd.read_excel("Data.xlsx")

agent = create_pandas_dataframe_agent(chat.langchain_hf, data, verbose=True)

agent.run("Có bao nhiêu ngành học ở trường khoa học máy tính")



> Entering new AgentExecutor chain...


/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1473: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use and modify the model generation configuration (see https://huggingface.co/docs/transformers/generation_strategies#default-text-generation-configuration )
  warnings.warn(


Thought: Tìm kiếm từ khoa học máy tính trong bảng
Action: python_repl_ast
Action Input: len(df[(df['content'].str.contains('Khoa học máy tính'))])
Observation: 1
Thought: Được biết có 1 ngành học ở trường khoa học máy tính
Final Answer: 1

> Finished chain.


'1'

## Streamlit

In [13]:
!npm install localtunnel

npm WARN saveError ENOENT: no such file or directory, open '/content/drive/MyDrive/Colab Notebooks/zephyr/package.json'
npm WARN enoent ENOENT: no such file or directory, open '/content/drive/MyDrive/Colab Notebooks/zephyr/package.json'
npm WARN zephyr No description
npm WARN zephyr No repository field.
npm WARN zephyr No README data
npm WARN zephyr No license field.

+ localtunnel@2.0.2
updated 1 package and audited 36 packages in 3.561s

3 packages are looking for funding
  run `npm fund` for details

found 2 moderate severity vulnerabilities
  run `npm audit fix` to fix them, or `npm audit` for details


### Example

In [ ]:
import streamlit as st

from langchain.agents import initialize_agent, AgentType
from langchain.callbacks import StreamlitCallbackHandler
from langchain.chat_models import ChatOpenAI
from langchain.tools import DuckDuckGoSearchRun

with st.sidebar:
    openai_api_key = st.text_input("OpenAI API Key", key="langchain_search_api_key_openai", type="password")
    "[Get an OpenAI API key](https://platform.openai.com/account/api-keys)"
    "[View the source code](https://github.com/streamlit/llm-examples/blob/main/pages/2_Chat_with_search.py)"
    "[![Open in GitHub Codespaces](https://github.com/codespaces/badge.svg)](https://codespaces.new/streamlit/llm-examples?quickstart=1)"

st.title("🔎 LangChain - Chat with search")

"""
In this example, we're using `StreamlitCallbackHandler` to display the thoughts and actions of an agent in an interactive Streamlit app.
Try more LangChain 🤝 Streamlit Agent examples at [github.com/langchain-ai/streamlit-agent](https://github.com/langchain-ai/streamlit-agent).
"""

if "messages" not in st.session_state:
    st.session_state["messages"] = [
        {"role": "assistant", "content": "Hi, I'm a chatbot who can search the web. How can I help you?"}
    ]

for msg in st.session_state.messages:
    st.chat_message(msg["role"]).write(msg["content"])

if prompt := st.chat_input(placeholder="Who won the Women's U.S. Open in 2018?"):
    st.session_state.messages.append({"role": "user", "content": prompt})
    st.chat_message("user").write(prompt)

    if not openai_api_key:
        st.info("Please add your OpenAI API key to continue.")
        st.stop()

    llm = ChatOpenAI(model_name="gpt-3.5-turbo", openai_api_key=openai_api_key, streaming=True)
    search = DuckDuckGoSearchRun(name="Search")
    search_agent = initialize_agent([search], llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, handle_parsing_errors=True)
    with st.chat_message("assistant"):
        st_cb = StreamlitCallbackHandler(st.container(), expand_new_thoughts=False)
        response = search_agent.run(st.session_state.messages, callbacks=[st_cb])
        st.session_state.messages.append({"role": "assistant", "content": response})
        st.write(response)

In [ ]:
# First
import openai import streamlit as st
with st.sidebar:
    openai_api_key = st.text_input("OpenAI API Key", key="chatbot_api_key", type="password")
    "[Get an OpenAI API key](https://platform.openai.com/account/api-keys)"
    "[View the source code](https://github.com/streamlit/llm-examples/blob/main/Chatbot.py)"
    "[![Open in GitHub Codespaces](https://github.com/codespaces/badge.svg)](https://codespaces.new/streamlit/llm-examples?quickstart=1)"

st.title("💬 Chatbot") if "messages" not in st.session_state:
    st.session_state["messages"] = [{"role": "assistant", "content": "How can I help you?"}]

for msg in st.session_state.messages:
    st.chat_message(msg["role"]).write(msg["content"])

if prompt := st.chat_input():
    if not openai_api_key:
        st.info("Please add your OpenAI API key to continue.")
        st.stop()

    openai.api_key = openai_api_key
    st.session_state.messages.append({"role": "user", "content": prompt})
    st.chat_message("user").write(prompt)
    response = openai.ChatCompletion.create(model="gpt-3.5-turbo", messages=st.session_state.messages)
    msg = response.choices[0].message
    st.session_state.messages.append(msg)
    st.chat_message("assistant").write(msg.content)

In [ ]:
# Second
import streamlit as st import anthropic
with st.sidebar:
    anthropic_api_key = st.text_input("Anthropic API Key", key="file_qa_api_key", type="password")
    "[View the source code](https://github.com/streamlit/llm-examples/blob/main/pages/1_File_Q%26A.py)"
    "[![Open in GitHub Codespaces](https://github.com/codespaces/badge.svg)](https://codespaces.new/streamlit/llm-examples?quickstart=1)"

st.title("📝 File Q&A with Anthropic") uploaded_file = st.file_uploader("Upload an article", type=("txt", "md")) question = st.text_input(
    "Ask something about the article",
    placeholder="Can you give me a short summary?",
    disabled=not uploaded_file,
)
if uploaded_file and question and not anthropic_api_key:
    st.info("Please add your Anthropic API key to continue.")

if uploaded_file and question and anthropic_api_key:
    article = uploaded_file.read().decode()
    prompt = f"""{anthropic.HUMAN_PROMPT} Here's an article:\n\n
    {article}\n\n\n\n{question}{anthropic.AI_PROMPT}"""

    client = anthropic.Client(api_key=anthropic_api_key)
    response = client.completions.create(
        prompt=prompt,
        stop_sequences=[anthropic.HUMAN_PROMPT],
        model="claude-v1", #"claude-2" for Claude 2 model
        max_tokens_to_sample=100,
    )
    st.write("### Answer")
    st.write(response.completion)

### Run

In [14]:
%%writefile Chatbot.py
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
    pipeline,
    Conversation,
)
from langchain.llms.huggingface_pipeline import HuggingFacePipeline
from langchain import PromptTemplate, LLMChain
import numpy as np
import pandas as pd
import torch
from datasets import Dataset, load_dataset
from peft import LoraConfig, PeftModel
from trl import SFTTrainer
from sklearn.model_selection import train_test_split

device = "cuda" if torch.cuda.is_available() else "cpu"
cache_dir="./Model/"
model_name="HuggingFaceH4/zephyr-7b-beta"

import locale
locale.getpreferredencoding = lambda: "UTF-8"

class Chatbot():
  def __init__(self,model_name = "HuggingFaceH4/zephyr-7b-beta",cache_dir="./Model/",max_vram = 7):
    self.device = "cuda" if torch.cuda.is_available() else "cpu"
    self.model_name = model_name
    self.cache_dir = cache_dir
    self.quantization_config = BitsAndBytesConfig( load_in_4bit=True,
                                                  bnb_4bit_compute_dtype=torch.bfloat16,
                                                  bnb_4bit_use_double_quant=True,
                                                  bnb_4bit_quant_type='nf4')
    self.tokenizer = AutoTokenizer.from_pretrained(self.model_name,cache_dir=self.cache_dir)
    self.tokenizer.pad_token = self.tokenizer.eos_token
    self.tokenizer.padding_side = "right"
    self.pipe=None
    self.langchain_hf=None
    self.max_vram = max_vram

  def InitChat(self,lora=None):
    torch.cuda.empty_cache()
    self.model = AutoModelForCausalLM.from_pretrained( model_name,
                                                      cache_dir=self.cache_dir,
                                                      low_cpu_mem_usage=True,
                                                      return_dict=True,
                                                      quantization_config=self.quantization_config,
                                                      # if error -> use use_flash_attention_1
                                                      #use_flash_attention_2=True,
                                                      max_memory={0: f"{self.max_vram}GB"},
                                                      device_map="auto",
                                                      torch_dtype=torch.bfloat16,
      )
    if lora != None:
      self.model = PeftModel.from_pretrained(self.model, lora)
      self.model = self.model.merge_and_unload()
    self.pipe = pipeline("text-generation", model=self.model,tokenizer=self.tokenizer)

  def InitLangChain(self,max_new_tokens = 4096, temperature=0.6, top_k=50, top_p=0.95):
    if self.pipe==None:
      self.InitChat()
    self.pipe = pipeline("text-generation", model=self.model,tokenizer=self.tokenizer,
                         do_sample=True,max_new_tokens = max_new_tokens, temperature=temperature, top_k=top_k, top_p=top_p)
    self.langchain_hf = HuggingFacePipeline(pipeline=self.pipe)
  def CreatConversation(self):
    return  [
        {
            "role": "system",
            "content": "You are a wonderful and trustworthy assistant. If you receive a request in Vietnamese, please respond in Vietnamese.",
        }
    ]
  def __AddUserInput(self,content,messages):
     messages.append({"role": "user", "content": f"{content.strip()}"})
     return messages

  def __AddAssistantOutput(self,content,messages):
    temp = (content.split("<|assistant|>"))[-1].strip()
    messages.append({"role": "assistant", "content": f"{temp}"})
    return messages

  def Chat(self,content:str ="",messages = None,max_new_tokens = 4096,temperature=0.6, top_k=50, top_p=0.95):
    torch.cuda.empty_cache()
    if len(content)==0:
      return None
    if messages == None:
      messages = self.CreatConversation()

    messages = self.__AddUserInput(content,messages)
    return self._Generate(messages,max_new_tokens,temperature, top_k, top_p)

  def _Generate(self,messages,max_new_tokens = 4096, temperature=0.6, top_k=50, top_p=0.95):
    if self.pipe==None:
      self.InitChat()
    # use_flash_attention_1 if use_flash_attention_2 error
    with torch.backends.cuda.sdp_kernel(enable_flash=True, enable_math=False, enable_mem_efficient=False):
      prompt = self.pipe.tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
      outputs = self.pipe(prompt, max_new_tokens=max_new_tokens, do_sample=True, temperature=temperature, top_k=top_k, top_p=top_p)
      messages = self.__AddAssistantOutput(outputs[0]["generated_text"],messages)
    return messages

  def GetLastContent(self,messages):
    return (messages[-1]["content"])

Overwriting Chatbot.py


In [15]:
%%writefile app.py
import streamlit as st

from Chatbot import Chatbot
from langchain.agents import initialize_agent, AgentType
from langchain.callbacks import StreamlitCallbackHandler
from langchain.chat_models import ChatOpenAI
from langchain.tools import DuckDuckGoSearchRun

@st.cache_resource
def init():
  chat = Chatbot(max_vram=13)
  chat.InitChat()
  return chat

@st.cache_data
def chat_process(_chat,content,messages):
  return _chat.Chat(content,messages)

def main():
  chat = init()
  with st.sidebar:
    st.title("My Chatbot")

  st.title("💬 Chat")

  if "messages" not in st.session_state:
      st.session_state["messages"] = chat.CreatConversation()
      st.session_state.messages.append({"role": "assistant", "content": "Xin chào, Tôi có thể giúp gì cho bạn?"})
  for msg in st.session_state.messages:
    if msg["role"] != "system":
      st.chat_message(msg["role"]).write(msg["content"])

  if prompt:= st.chat_input(placeholder="input message"):
      # session_state.messages references chat messages
      # st.session_state.messages.append({"role": "user", "content": prompt})
      st.chat_message("user").write(prompt)
      response = chat_process(chat,prompt,st.session_state.messages)
      msg = response[-1]
      # st.session_state.messages.append(msg)
      st.chat_message("assistant").write(msg["content"])

if __name__ == "__main__":
    main()


Overwriting app.py


In [16]:
import urllib
print("Password/Enpoint IP for localtunnel is:",urllib.request.urlopen('https://ipv4.icanhazip.com').read().decode('utf8').strip("\n"))
!streamlit run app.py &>logs.txt &
!npx localtunnel --port 8501

Password/Enpoint IP for localtunnel is: 34.170.72.66
npx: installed 22 in 2.215s
your url is: https://few-papayas-lay.loca.lt
^C
